In [6]:
from tensorboard.backend.event_processing import event_accumulator
import pandas as pd
from pathlib import Path


In [126]:
from tbparse import SummaryReader


def get_metric_results_from_eventfile(dirpath, metrics, save_path=""):
    reader = SummaryReader(dirpath, pivot=False, extra_columns={'dir_name'})
    df = reader.scalars
    df = df.rename(columns={"tag": "metric", "dir_name": "fold"})
    df = df.groupby(["fold","metric"])['value'].max().reset_index()
    df = df[df["metric"].isin(metrics)].set_index(["fold"]).sort_values("metric").reset_index()
    res = df.pivot(index='fold', columns='metric', values='value').rename_axis(columns=None)
    if save_path:
        res.to_markdown(save_path, mode="a", tablefmt="github", index=True)
    return res

In [129]:
metrics = ["train_auroc", "val_auroc", "train_accuracy_epoch", "val_accuracy_epoch"]
save_path = "/Users/rickgentry/emotive_lab/eyemind/results/comprehension_results.md"
get_metric_results_from_eventfile("/Users/rickgentry/emotive_lab/eyemind/lightning_logs/informer_MW", metrics, save_path)

,train_accuracy_epoch,train_auroc,val_accuracy_epoch,val_auroc
fold,,,,
fold0,0.616071,0.663869,0.585938,0.568236
fold1,0.741071,0.814575,0.679688,0.725127
fold2,0.625000,0.661614,0.593750,0.554024
fold3,0.604911,0.617771,0.609375,0.567321
